This notebook is the one I used to test the port from the original repository's code into something I could use. As such, it uses the standard chair data and configuration settings.

# Train

In [1]:
import os
import random
import numpy as np
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
from torch.autograd import Variable
from datasets import PartDataset
from pointnet import PointNetCls, PointGen

In [2]:
batchSize = 32
nepoch = 25
num_points = 2500
manualSeed = random.randint(1, 10000)
print('Seed is %i' % manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

# For data loading?
workers = 4

# Output folder
outf = 'benchmark/models'

# Model path
model = ''

Seed is 8302


In [4]:
data_folder = 'benchmark/data'

dataset = PartDataset(root=data_folder,
                      class_choice=['Chair'],
                      classification=True)
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batchSize,
                                         shuffle=True,
                                         num_workers=workers)

test_dataset = PartDataset(root=data_folder,
                           class_choice=['Chair'],
                           classification=True,
                           train=False)
testdataloader = torch.utils.data.DataLoader(test_dataset,
                                             batch_size=batchSize,
                                             shuffle=True,
                                             num_workers=workers)

cudnn.benchmark = True

print('Training size of %i' % len(dataset))
print('Test size of %i' % len(test_dataset))
print('%i class[es]' % len(dataset.classes))

Training size of 3371
Test size of 375
1 class[es]


In [5]:
try:
    os.makedirs(outf)
except OSError:
    pass

classifier = PointNetCls(k=2, num_points=num_points)
gen = PointGen(num_points=num_points)

if model != '': 
    classifier.load_state_dict(torch.load(model))

print(classifier)
print(gen)

PointNetCls(
  (feat): PointNetfeat(
    (stn): STN3d(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
    )
    (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Lin

In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1: 
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1: 
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [7]:
classifier.apply(weights_init)
gen.apply(weights_init)

classifier.cuda()
gen.cuda()

optimizerD = optim.Adagrad(classifier.parameters(), lr=0.001)
optimizerG = optim.Adagrad(gen.parameters(), lr=0.001)

num_batch = len(dataset)/batchSize

In [8]:
for epoch in range(nepoch):
    for i, data in enumerate(dataloader): 
        optimizerD.zero_grad()
        points, _ = data
        points = Variable(points)

        bs = points.size()[0]
        target = Variable(torch.from_numpy(np.ones(bs,).astype(np.int64))).cuda()
        points = points.transpose(2,1)
        points = points.cuda()

        pred, trans = classifier(points)
        loss1 = F.nll_loss(pred, target)
        
        sim_noise = Variable(torch.randn(bs, 100)).cuda()
        fake = gen(sim_noise)
        fake_target = Variable(torch.from_numpy(np.zeros(bs,).astype(np.int64))).cuda()
        pred2, trans2 = classifier(fake)

        loss2 = F.nll_loss(pred2, fake_target)
        lossD = (loss1 + loss2)/2
        lossD.backward()

        optimizerD.step()
        optimizerG.zero_grad()

        sim_noise = Variable(torch.randn(bs, 100)).cuda()
        points = gen(sim_noise)
        pred, trans = classifier(points)
        target = Variable(torch.from_numpy(np.ones(bs,).astype(np.int64))).cuda()
        lossG = F.nll_loss(pred, target)
        lossG.backward()
        optimizerG.step()

        print('[%d: %d/%d] train lossD: %f lossG: %f' % (epoch, i, num_batch,
                                                         lossD.data[0],
                                                         lossG.data[0]))

    torch.save(classifier.state_dict(), '%s/modelD_%d.pth' % (outf, epoch))
    torch.save(gen.state_dict(), '%s/modelG_%d.pth' % (outf, epoch))

pointnet.py:97: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x), trans
/home/ktran/miniconda3/envs/art/lib/python2.7/site-packages/ipykernel_launcher.py:36: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/ktran/miniconda3/envs/art/lib/python2.7/site-packages/ipykernel_launcher.py:37: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[0: 0/105] train lossD: 0.713605 lossG: 7.314999
[0: 1/105] train lossD: 2.048188 lossG: 0.053237
[0: 2/105] train lossD: 1.640482 lossG: 1.738608
[0: 3/105] train lossD: 0.676649 lossG: 0.503927
[0: 4/105] train lossD: 0.714342 lossG: 2.154416
[0: 5/105] train lossD: 0.718418 lossG: 0.634470
[0: 6/105] train lossD: 0.638660 lossG: 1.653548
[0: 7/105] train lossD: 0.587056 lossG: 0.619641
[0: 8/105] train lossD: 0.626860 lossG: 1.777450
[0: 9/105] train lossD: 0.639366 lossG: 0.499385
[0: 10/105] train lossD: 0.677799 lossG: 1.793209
[0: 11/105] train lossD: 0.668823 lossG: 0.601111
[0: 12/105] train lossD: 0.649148 lossG: 1.413018
[0: 13/105] train lossD: 0.620169 lossG: 0.697971
[0: 14/105] train lossD: 0.639743 lossG: 1.110772
[0: 15/105] train lossD: 0.641848 lossG: 0.593196
[0: 16/105] train lossD: 0.663278 lossG: 1.410146
[0: 17/105] train lossD: 0.690792 lossG: 0.536878
[0: 18/105] train lossD: 0.695418 lossG: 1.165003
[0: 19/105] train lossD: 0.660217 lossG: 0.675089
[0: 20/105

[1: 60/105] train lossD: 0.427921 lossG: 2.333000
[1: 61/105] train lossD: 0.418588 lossG: 0.936868
[1: 62/105] train lossD: 0.512526 lossG: 3.245042
[1: 63/105] train lossD: 0.661913 lossG: 0.803773
[1: 64/105] train lossD: 0.553845 lossG: 2.690171
[1: 65/105] train lossD: 0.479630 lossG: 1.090888
[1: 66/105] train lossD: 0.385781 lossG: 2.229141
[1: 67/105] train lossD: 0.337654 lossG: 1.233967
[1: 68/105] train lossD: 0.352057 lossG: 2.046432
[1: 69/105] train lossD: 0.298094 lossG: 1.446527
[1: 70/105] train lossD: 0.336204 lossG: 2.099096
[1: 71/105] train lossD: 0.257079 lossG: 1.577073
[1: 72/105] train lossD: 0.263108 lossG: 2.423653
[1: 73/105] train lossD: 0.362017 lossG: 0.998476
[1: 74/105] train lossD: 0.368053 lossG: 3.026442
[1: 75/105] train lossD: 0.506320 lossG: 1.153632
[1: 76/105] train lossD: 0.310525 lossG: 2.575317
[1: 77/105] train lossD: 0.295425 lossG: 1.692006
[1: 78/105] train lossD: 0.257109 lossG: 2.258981
[1: 79/105] train lossD: 0.254510 lossG: 1.723472


[3: 14/105] train lossD: 0.140954 lossG: 1.923136
[3: 15/105] train lossD: 0.126394 lossG: 3.597544
[3: 16/105] train lossD: 0.098628 lossG: 2.903117
[3: 17/105] train lossD: 0.114946 lossG: 3.170691
[3: 18/105] train lossD: 0.103216 lossG: 2.873642
[3: 19/105] train lossD: 0.128540 lossG: 2.937294
[3: 20/105] train lossD: 0.119103 lossG: 2.612145
[3: 21/105] train lossD: 0.172597 lossG: 3.830541
[3: 22/105] train lossD: 0.099853 lossG: 3.045916
[3: 23/105] train lossD: 0.147923 lossG: 2.172693
[3: 24/105] train lossD: 0.175083 lossG: 4.472316
[3: 25/105] train lossD: 0.231187 lossG: 1.860482
[3: 26/105] train lossD: 0.157392 lossG: 3.707970
[3: 27/105] train lossD: 0.206445 lossG: 1.843218
[3: 28/105] train lossD: 0.122877 lossG: 3.353858
[3: 29/105] train lossD: 0.216589 lossG: 1.927034
[3: 30/105] train lossD: 0.310490 lossG: 5.639345
[3: 31/105] train lossD: 0.611145 lossG: 0.484725
[3: 32/105] train lossD: 0.674165 lossG: 5.877465
[3: 33/105] train lossD: 1.179200 lossG: 2.576397


[4: 73/105] train lossD: 0.544128 lossG: 2.543267
[4: 74/105] train lossD: 0.523764 lossG: 0.980637
[4: 75/105] train lossD: 0.503971 lossG: 1.087269
[4: 76/105] train lossD: 0.339291 lossG: 1.731305
[4: 77/105] train lossD: 0.402474 lossG: 1.053282
[4: 78/105] train lossD: 0.429216 lossG: 1.932373
[4: 79/105] train lossD: 0.427296 lossG: 0.866420
[4: 80/105] train lossD: 0.387002 lossG: 1.581046
[4: 81/105] train lossD: 0.375251 lossG: 1.057954
[4: 82/105] train lossD: 0.425195 lossG: 1.855121
[4: 83/105] train lossD: 0.416892 lossG: 1.007687
[4: 84/105] train lossD: 0.433613 lossG: 2.447027
[4: 85/105] train lossD: 0.611620 lossG: 0.549181
[4: 86/105] train lossD: 0.475178 lossG: 2.161495
[4: 87/105] train lossD: 0.536140 lossG: 0.894283
[4: 88/105] train lossD: 0.455053 lossG: 1.777207
[4: 89/105] train lossD: 0.490349 lossG: 0.818033
[4: 90/105] train lossD: 0.481525 lossG: 1.920864
[4: 91/105] train lossD: 0.444701 lossG: 0.796809
[4: 92/105] train lossD: 0.425381 lossG: 1.865071


...

[18: 53/105] train lossD: 0.294456 lossG: 3.865735
[18: 54/105] train lossD: 0.417345 lossG: 1.049001
[18: 55/105] train lossD: 0.288252 lossG: 3.673310
[18: 56/105] train lossD: 0.381985 lossG: 0.910653
[18: 57/105] train lossD: 0.331131 lossG: 3.922314
[18: 58/105] train lossD: 0.509188 lossG: 1.283979
[18: 59/105] train lossD: 0.362464 lossG: 3.866815
[18: 60/105] train lossD: 0.531890 lossG: 1.277147
[18: 61/105] train lossD: 0.307225 lossG: 3.292402
[18: 62/105] train lossD: 0.265680 lossG: 1.673300
[18: 63/105] train lossD: 0.255777 lossG: 2.637062
[18: 64/105] train lossD: 0.241976 lossG: 1.120261
[18: 65/105] train lossD: 0.234718 lossG: 3.189334
[18: 66/105] train lossD: 0.177533 lossG: 2.013332
[18: 67/105] train lossD: 0.169964 lossG: 1.853394
[18: 68/105] train lossD: 0.150051 lossG: 2.344090
[18: 69/105] train lossD: 0.209608 lossG: 1.482048
[18: 70/105] train lossD: 0.217478 lossG: 3.036017
[18: 71/105] train lossD: 0.130624 lossG: 2.261895
[18: 72/105] train lossD: 0.195

[20: 2/105] train lossD: 0.148273 lossG: 2.631329
[20: 3/105] train lossD: 0.155167 lossG: 2.207781
[20: 4/105] train lossD: 0.158367 lossG: 2.450618
[20: 5/105] train lossD: 0.152202 lossG: 2.553001
[20: 6/105] train lossD: 0.182366 lossG: 2.367412
[20: 7/105] train lossD: 0.152621 lossG: 2.124619
[20: 8/105] train lossD: 0.174765 lossG: 2.502384
[20: 9/105] train lossD: 0.178680 lossG: 1.930770
[20: 10/105] train lossD: 0.201938 lossG: 3.030567
[20: 11/105] train lossD: 0.179873 lossG: 1.926160
[20: 12/105] train lossD: 0.190116 lossG: 2.622515
[20: 13/105] train lossD: 0.230617 lossG: 1.612082
[20: 14/105] train lossD: 0.243182 lossG: 3.269634
[20: 15/105] train lossD: 0.216094 lossG: 1.627549
[20: 16/105] train lossD: 0.211108 lossG: 2.752710
[20: 17/105] train lossD: 0.166568 lossG: 1.862046
[20: 18/105] train lossD: 0.171757 lossG: 2.770157
[20: 19/105] train lossD: 0.124549 lossG: 2.143469
[20: 20/105] train lossD: 0.150861 lossG: 2.576698
[20: 21/105] train lossD: 0.166010 loss

[21: 58/105] train lossD: 0.166613 lossG: 2.368094
[21: 59/105] train lossD: 0.217093 lossG: 2.049776
[21: 60/105] train lossD: 0.241279 lossG: 3.652853
[21: 61/105] train lossD: 0.343596 lossG: 0.743030
[21: 62/105] train lossD: 0.312669 lossG: 3.976662
[21: 63/105] train lossD: 0.360972 lossG: 1.949224
[21: 64/105] train lossD: 0.131028 lossG: 2.646037
[21: 65/105] train lossD: 0.173110 lossG: 1.911351
[21: 66/105] train lossD: 0.149866 lossG: 2.696773
[21: 67/105] train lossD: 0.185093 lossG: 1.964043
[21: 68/105] train lossD: 0.247160 lossG: 1.769228
[21: 69/105] train lossD: 0.229262 lossG: 2.905524
[21: 70/105] train lossD: 0.206053 lossG: 2.032660
[21: 71/105] train lossD: 0.170889 lossG: 2.276565
[21: 72/105] train lossD: 0.227299 lossG: 1.536859
[21: 73/105] train lossD: 0.248325 lossG: 2.495476
[21: 74/105] train lossD: 0.167503 lossG: 2.077356
[21: 75/105] train lossD: 0.219272 lossG: 1.985125
[21: 76/105] train lossD: 0.148581 lossG: 2.117798
[21: 77/105] train lossD: 0.171

[23: 8/105] train lossD: 0.167735 lossG: 2.537544
[23: 9/105] train lossD: 0.220936 lossG: 2.624563
[23: 10/105] train lossD: 0.183968 lossG: 2.147885
[23: 11/105] train lossD: 0.190030 lossG: 2.405313
[23: 12/105] train lossD: 0.363736 lossG: 3.126443
[23: 13/105] train lossD: 0.234319 lossG: 1.975768
[23: 14/105] train lossD: 0.340436 lossG: 3.490463
[23: 15/105] train lossD: 0.421069 lossG: 0.917634
[23: 16/105] train lossD: 0.422950 lossG: 4.459209
[23: 17/105] train lossD: 0.514462 lossG: 1.808867
[23: 18/105] train lossD: 0.189220 lossG: 2.632934
[23: 19/105] train lossD: 0.222871 lossG: 1.866238
[23: 20/105] train lossD: 0.320132 lossG: 2.388358
[23: 21/105] train lossD: 0.213756 lossG: 2.175523
[23: 22/105] train lossD: 0.299927 lossG: 1.778670
[23: 23/105] train lossD: 0.173022 lossG: 2.685077
[23: 24/105] train lossD: 0.163463 lossG: 2.197710
[23: 25/105] train lossD: 0.187978 lossG: 2.279108
[23: 26/105] train lossD: 0.292227 lossG: 1.564877
[23: 27/105] train lossD: 0.27563

[24: 64/105] train lossD: 0.333640 lossG: 1.556478
[24: 65/105] train lossD: 0.153761 lossG: 2.750573
[24: 66/105] train lossD: 0.246306 lossG: 1.388611
[24: 67/105] train lossD: 0.269371 lossG: 3.676933
[24: 68/105] train lossD: 0.437772 lossG: 1.520380
[24: 69/105] train lossD: 0.208728 lossG: 3.203803
[24: 70/105] train lossD: 0.161737 lossG: 2.199029
[24: 71/105] train lossD: 0.174909 lossG: 2.086774
[24: 72/105] train lossD: 0.198922 lossG: 2.693957
[24: 73/105] train lossD: 0.214388 lossG: 1.642923
[24: 74/105] train lossD: 0.355909 lossG: 3.676988
[24: 75/105] train lossD: 0.327907 lossG: 1.538867
[24: 76/105] train lossD: 0.255690 lossG: 3.324491
[24: 77/105] train lossD: 0.198083 lossG: 2.330711
[24: 78/105] train lossD: 0.132608 lossG: 2.072622
[24: 79/105] train lossD: 0.192467 lossG: 2.220035
[24: 80/105] train lossD: 0.155727 lossG: 2.320534
[24: 81/105] train lossD: 0.122957 lossG: 2.336127
[24: 82/105] train lossD: 0.176577 lossG: 2.224694
[24: 83/105] train lossD: 0.162

# Generate

In [10]:
i_ = 50
j_ = 3
k_ = 100

sim_noise = Variable(torch.randn(j_, k_)).cuda()
sim_noises = Variable(torch.zeros(i_*j_, k_)).cuda()

for j in range(j_):
    for i in range(i_):
        x = (1-i/i_)
        sim_noises[i + i_*j] = (sim_noise[j]*x + 
                                sim_noise[(j+1) % j_] * (1-x))

points = gen(sim_noises)
points_np = points.cpu().transpose(2, 1).data.numpy()
print(points_np.shape)

(150, 2500, 3)


# View
Let's test MY viewing method

In [11]:
import ase
from ase.visualize import view

In [12]:
image_number = random.randint(0, points_np.shape[0])
print('Randomly selected cloud number %i' % image_number)

positions = points_np[image_number, :, :]
n_atoms = positions.shape[0]
atoms = ase.Atoms(['H']*n_atoms, positions)
view(atoms, viewer='x3d')

Randomly selected cloud number 79
